In [ ]:
import numpy as np
import h5py
from matplotlib import pyplot as plt

f = h5py.File('D:\\fastMRI_DATA\\singlecoil_val\\file1000000.h5','r')
# content of the single oil
f.keys()

In [ ]:
print(f['reconstruction_esc'].shape)
# choose a slice
i_img = 8
# reconstruction_rss is the original milti-coil image from which the single-coil k-space was generated
# truth = f['reconstruction_rss'][i_img]
# reconstruction_esc is the fully sampled image from the single-coil k-space
truth = f['reconstruction_esc'][i_img]

In [ ]:
f.filename

In [ ]:
# how to fourrier transform
kdata = f['kspace'][i_img]
kdata = np.fft.ifftshift(kdata)
recon_img = np.fft.ifft2(kdata)
recon_img = np.fft.fftshift(recon_img)
plt.imshow(abs(recon_img),cmap='gray')

In [ ]:
# images in the files are size 320x320 taken at center 
print(recon_img.shape)
print(truth.shape)

In [ ]:
# re-size recon image
xstart = int((recon_img.shape[0]-320)/2)
xend = int(recon_img.shape[0]-xstart)
ystart = int((recon_img.shape[1]-320)/2)
yend = int(recon_img.shape[1]-ystart)
print(xstart,xend,ystart,yend)
recon_windowed = recon_img[xstart:xend,ystart:yend]

In [ ]:
plt.imshow(abs(recon_windowed),cmap='gray')

In [ ]:
# There is a factor of ~485.3040 between our recon and images in the file...
factor = np.amax(f['reconstruction_esc'][i_img])/np.amax(abs(recon_windowed))
np.isclose(factor,485.3040)
plt.imshow(abs(recon_windowed)*factor,cmap='gray')

In [ ]:
# now a multi-coil
mf = h5py.File('D:\\fastMRI_DATA\\multicoil_train\\file1000045.h5','r')
mf.keys()

In [ ]:
mf['kspace'].shape
# it goes (# of slices, # of coils, # pixels, #pixels)

In [ ]:
from utils.fastMRI_utils import ifft, crop, rss
imgs = []
for slice in mf['kspace']:
    tmp_imgs = []
    for coil in slice:
        img = ifft(coil)
        img = crop(img)
        tmp_imgs.append(img)
    # img = np.zeros(tmp_imgs[0].shape,dtype=np.complex128)
    imgs.append(rss(np.array(tmp_imgs)))
imgs = np.array(imgs)

In [ ]:
bools = []
for i, img in enumerate(imgs):
    bools.append(np.isclose(np.amax(img)*485.3040174057125,np.amax(mf['reconstruction_rss'][i])))
all(bools)

In [ ]:
plt.imshow(imgs[15],cmap='gray')

In [ ]:
plt.imshow(mf['reconstruction_rss'][15],cmap='gray')

In [ ]:
np.amax(mf['reconstruction_rss'][15])/np.amax(imgs[15])